# Step 1: Install and import libraries

In [5]:
!pip install python-dotenv
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=21cde7bd301aa7d4cc1732d4cb6ef8dd35f4b07db965b8959c5114d287ffc5f9
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [6]:
import os
from openai import OpenAI
from datetime import datetime
from dotenv import load_dotenv # pip install python-dotenv
import os
import whisper # pip install -U openai-whisper

# Step2: Mount Google drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
shared_folder_path = '/content/drive/My Drive/LLM_Negotiation_Simulator'

In [14]:
# Test path
file_path = f'{shared_folder_path}/prompts/demo_simulation_scenario_short.txt'

if os.path.exists(file_path):
    print("File found!")
else:
    print("File not found.")


File found!


In [20]:
env_path = f'{shared_folder_path}/.env'
load_dotenv(env_path)
openai_api_key = os.getenv("API_KEY")

# Step3: Load Functions

In [56]:
temp = 0.8
def simulation_setup():
    print("Hi, I'm a Negotiation Simulator. Please provide a description of the negotiation case:")
    simulation_description = input()
    if len(simulation_description) == 0:
        with open(f'{shared_folder_path}/prompts/demo_simulation_scenario_short.txt', 'r') as file:
            print("Description of the negotiation case is successfully read in.")
            simulation_description = file.readline().strip()

    print("In this negotiation case, which side is the negotiator who wants to de-escalate and which side is the adversarial party who has strong demands or may have strong resistance? Just give me the two roles in the format A, B in the order negotiator, adversarial party:")
    simulation_roles = input().split(", ")
    simulation_negotiator = simulation_roles[0]
    simulation_counterparty = simulation_roles[1]

    print("Do you want to upload a dialogue for feedback or do you want to start a simulation? Reply 'Feedback' or 'Simulation'")
    correctly_input_simulation_or_feedback = False

    while not correctly_input_simulation_or_feedback:
        simulation_or_feedback = input()
        if simulation_or_feedback == 'Feedback':
            print("You choose to upload a dialogue.")
            correctly_input_simulation_or_feedback = True
            simulation_user_role = None

        elif simulation_or_feedback == 'Simulation':
            print("You choose to start a simulation.")
            correctly_input_simulation_or_feedback = True

            # prompt for user role
            print("In this negotiation case, which role do you want to play? Reply 'Negotiator' or 'Adversarial Party'")
            correctly_input_simulation_user_role = False

            while not correctly_input_simulation_user_role:
                simulation_user_role = input()
                if simulation_user_role == 'Negotiator':
                    print("You choose to play as the Negotiator. The negotiation starts now.")
                    correctly_input_simulation_user_role = True
                elif simulation_user_role == 'Adversarial Party':
                    print("You choose to play as the Adversarial Party. The negotiation starts now.")
                    correctly_input_simulation_user_role = True
                else:
                    print("Please reply 'Negotiator' or 'Adversarial Party' to choose the role you want to play:")
        else:
            print("Please reply 'Feedback' or 'Simulation' to choose what you want me to do:")

    return simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role

def run_simulation(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role):
    if simulation_user_role == 'Negotiator':
        run_gpt_as_counterparty(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role)
    elif simulation_user_role == 'Adversarial Party':
        run_gpt_as_negotiator(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role)


def run_gpt_as_counterparty(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role):
    error = False
    conversation = []
    stages = [1]
    strategy_correctness = []

    # initial_prompt
    initial_prompt = "You are a GPT designed to simulate a hostile or upset Adversarial Party to help users practice de-escalation and negotiation skills. You will engage in interactive conversations where the user applies specific de-escalation strategies based on the adversarial party’s level of escalation. The conversation will progress through six stages, and the user will practice responding appropriately based on the adversarial party’s current behavior."

    # read in strategy
    with open(f"{shared_folder_path}/prompts/simulation_strategy.txt", "r") as file:
        content = file.read()
    strategy = "Here is a description of the 6 different stages the aggressive Adversarial Party could be at during a negotiation, and the correct strategy a professional negotiator should take for each stage, in order to de-escalate the situation and start to work towards common understanding and solutions:"
    strategy = (strategy + '\n'+ content)

    # sample script
    with open(f"{shared_folder_path}/prompts/simulation_sample_script.txt", "r") as file:
        content = file.read()
    sample_script = "Here is an example of the 6 stages negotiation. Use this as a referece when generating the Adversarial Party's response at certain stage:"
    sample_script = (sample_script + '\n'+ content)

    # simulation
    stage = 1
    # save the simulation dialogue
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"simulation_log_{timestamp}.txt"
    output_dir = f"{shared_folder_path}/simulation_dialogues/"
    file_path = os.path.join(output_dir, file_name)
    first_time = True
    with open(file_path, "w") as file:
        while stage < 7 and len(stages) <=15 and not error:
            # print("Current stage:")
            # print(stage)
            if stage == 1:
                if first_time:
                    first_time = False
                    scenario_and_context = (
                        f"Now user has input a scenario: {simulation_description}\n"
                        f"User chose to play as {simulation_user_role}, which is {simulation_negotiator}. "
                        f"Now simulate a confrontational or aggressive Adversarial Party, who is {simulation_counterparty}, "
                        f"and start the conversation at stage {stage}. Please only output the dialogue of the Adversarial Party that you act without other explanation."
                    )
                else:
                    scenario_and_context = (
                        f"Now user has input a scenario: {simulation_description}\n"
                        f"User chose to play as {simulation_user_role}, which is {simulation_negotiator}. "
                        f"This is the conversation history:\n{conversation_history}.\n"
                        f"Now simulate a confrontational or aggressive Adversarial Party, who is {simulation_counterparty}, "
                        f"and continue the conversation at stage {stage}. You don't need to greet again. Please only output the dialogue of the Adversarial Party that you act without other explanation."
                    )
            else:
                conversation_history = '\n'.join(conversation)
                scenario_and_context = (
                    f"Now user has input a scenario: {simulation_description}\n"
                    f"User chose to play as {simulation_user_role}, which is {simulation_negotiator}. "
                    f"This is the conversation history:\n{conversation_history}.\n"
                    f"Now simulate a confrontational or aggressive Adversarial Party, who is {simulation_counterparty}, "
                    f"and continue the above conversation at stage {stage}. Please only output the dialogue of the Adversarial Party that you act without other explanation."
                )

            # Counterparty response
            gpt_prompt = initial_prompt + '\n' + strategy + '\n' + sample_script + '\n' + scenario_and_context
            gpt_response = call_gpt(gpt_prompt,temp)
            print("Adversarial Party:")
            print(gpt_response)
            if gpt_response == "error":
                error = True
                break
            counter_party_response = f'Adversarial Party({simulation_counterparty}): ' + gpt_response
            conversation.append(f"Conversation{len(stages)}:\n")
            conversation.append(counter_party_response)
            file.write(f"Stage {stage} - Adversarial Party:\n{counter_party_response}\n")

            # Negotiator response
            print("Negotiator:")
            user_response = input()
            conversation.append(f'Negotiator({simulation_negotiator}): ' + user_response)
            file.write(f"Negotiator:\n{user_response}\n")

            # Check strategy
            conversation_history = '\n'.join(conversation)
            is_correct_strategy = check_strategy(user_response, conversation_history, stage) # 1 or 0
            if is_correct_strategy == "error":
                error = True
                print("An error occurred while checking strategy.")
                break
            if is_correct_strategy == 1:
                stage += 1
            correct_status = "Correct\n" if is_correct_strategy == 1 else "Incorrect\n"
            file.write(f"Correct Status: {correct_status}\n")
            stages.append(stage)
            strategy_correctness.append(is_correct_strategy)
            # print(f"{'Correct' if is_correct_strategy == 1 else 'Incorrect'}\n")


    # Generate feedback

    scenario_and_context = (
                f"Here is the negotiation scenario: {simulation_description}\n"
                f"User chose to play as {simulation_user_role}, which is {simulation_negotiator}. "
                f"Now you have finished a negotiation simulation, and this is the conversation history:\n{conversation_history}.\n"
            )

    with open(f"{shared_folder_path}/prompts/feedback_template.txt", "r") as file:
        sample_feedback = file.read()

    feedback_prompt = initial_prompt + '\n' + strategy + '\n' + scenario_and_context
    feedback_prompt += (
        "We recorded the stage of the the adversarial party at each round of the conversation and whether the negotiator responded with the correct strategy. "
        "Now please generate feedback for the negotiator based on their responses at each stage. "
        "Here is the summary of their performance for each round of conversation:\n"
    )

    for idx, (i, j) in enumerate(zip(stages, strategy_correctness)):
        feedback_prompt += f"Conversation {idx+1}: Stage:{i}, If the strategy is correct: {j}\n"

    feedback_prompt += (
        "Based on this performance, provide constructive feedback on areas where the negotiator did well, "
        "and where they could improve. For each round of conversation, comment on 'What you did well' and 'Improvements'. Focus more on stages where the negotiator used incorrect strategies if there were any."
        f"Here is an example feedback for conversation 2, where the conversation is at stage 3: {sample_feedback}. You should follow the same format.\n"
        "After generating summary for each round of conversation, generate an overall feedback on the entire dialogue. An example: Overall, the negotiator applied de-escalation strategies effectively, especially in capturing emotions and promoting collaboration. However, there is room for more explicit alignment with emotions and strategic focus at each stage to strengthen the negotiation process."
    )
    feedback = call_gpt(feedback_prompt,temp)
    if stage > 6:
        prefix_feedback = "Congratulations! You did a great job and successfully passed all stages using the correct strategies. Here's some feedback for your reference:\n"
    else:
        prefix_feedback = "Thank you for practicing. However, you didn’t manage to pass all stages with the correct strategies within the 15 rounds of conversation. Here's some feedback for your reference:\n"
    print(prefix_feedback)
    print(feedback)

    file_name = f"simulation_log_{timestamp}.txt"
    output_dir = f"{shared_folder_path}/simulation_dialogues/"
    file_path = os.path.join(output_dir, file_name)
    final_feedback = prefix_feedback + feedback
    with open(file_path, "a") as file:
        file.write("\nFinal Feedback:\n")
        file.write(final_feedback + "\n")
    print(f"\nSimulation completed. Simulation script is saved as {file_name}")

    # prompt user to practice again if any incorrect strategy was used
    if 0 in strategy_correctness:
        print("Since you did not use the correct strategy for some stages, would you like to practice the simulation again? (yes/no): ")
        correctly_input_practice_again = False
        while not correctly_input_practice_again:
            practice_again = input().lower()
            if practice_again == 'yes':
                print("\nYou choose to practice the negotation again. The negotiation starts now.")
                run_simulation(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role)
                correctly_input_practice_again = True
            elif practice_again == 'no':
                print("\nThank you for practicing. You can come back anytime to practice your skills.")
                correctly_input_practice_again = True
            else:
                print("\nPlease reply 'yes' or 'no' to choose to practice again or not:")
    else:
        print("\nThank you for practicing. You can come back anytime to practice your skills.")

def run_gpt_feedback(simulation_description, simulation_negotiator, simulation_counterparty):
    print("Do you want to provide the negotiation dialogue as text or upload an MP3 recording? Reply 'Text' or 'MP3'.")
    correctly_input_text_or_audio = False
    while not correctly_input_text_or_audio:
        text_or_audio = input()
        if text_or_audio == 'Text':
            print("\nYou choose to provide the dialogue as text.")
            correctly_input_text_or_audio = True
        elif text_or_audio == 'MP3':
            print("\nYou choose to provide the dialogue as MP3.")
            correctly_input_text_or_audio = True
        else:
            print("\nPlease reply 'Text' or 'MP3' to choose how you want to provide the dialogue:")

    if text_or_audio == 'Text':
        print("Please input the dialogue of the negotiation simulation for feedback:")
        input_dialogue = input()

        if len(input_dialogue) == 0:
            with open(f'{shared_folder_path}/prompts/example-dialogue-1.txt', 'r') as file:
                print("Dialogue is successfully read in.")
                input_dialogue = file.readline().strip()

    if text_or_audio == 'MP3':
        print("Please input the file path for the MP3 file:")
        mp3_file_path = input()

        while not os.path.exists(mp3_file_path) or not mp3_file_path.endswith('.mp3'):
            print("Invalid file path or file type. Please ensure the file exists and is an MP3. Please input file path:")
            mp3_file_path = input()
        print("File path successfully uploaded.\n")

        print("Processing audio...\n")
        input_dialogue = call_whisper(mp3_file_path)

        print("Here is the transcript of the dialogue you provided: \n")
        print(input_dialogue)

        # Save the transcribed dialogue
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        transcribed_file_name = f"transcribed_dialogue_{timestamp}.txt"
        output_dir = f"{shared_folder_path}/transcriptions/"
        os.makedirs(output_dir, exist_ok=True)
        transcribed_file_path = os.path.join(output_dir, transcribed_file_name)
        with open(transcribed_file_path, "w") as file:
            file.write(input_dialogue)
        print(f"\nTranscribed dialogue saved as {transcribed_file_name}.")

    # initial_prompt
    initial_prompt = "You are a GPT designed to evaluate negotiation skills, especially de-escalation strategies."

    # read in strategy
    with open(f"{shared_folder_path}/prompts/simulation_strategy.txt", "r") as file:
        content = file.read()
    strategy = "Here is a description of the 6 different stages the aggressive adversarial party could be at during a negotiation, and the correct strategy a professional negotiator should take for each stage, in order to de-escalate the situation and start to work towards common understanding and solutions:"
    strategy = (strategy + '\n'+ content)

    scenario_and_context = (
                f"Here is the negotiation scenario: {simulation_description}\n"
                f"In this scenario, the negotiator is the {simulation_negotiator} and the aggressive adversarial party is the {simulation_counterparty}. "
                f"Now here is the dialogue of a negotiation simulation you need to evaluate. Each exchange between the negotiator and the adversarial party is considered a conversation round: {input_dialogue}\n"
                f"Pay special attention to which part is said by which party if not marked clearly.\n"
            )

    with open(f"{shared_folder_path}/prompts/feedback_template.txt", "r") as file:
        sample_feedback = file.read()

    feedback_prompt = initial_prompt + '\n' + strategy + '\n' + scenario_and_context

    feedback_prompt += (
        "Now, please provide feedback on how well the negotiator applied de-escalation strategies based on the six stages.\n"
        "For each stage, comment on 'What you did well' and 'Improvements needed'. Focus on specific stages and strategies. Do not start with the agreement to perform this task. "
        f"Here is an example feedback for conversation 2, where the conversation is at stage 3: {sample_feedback}. You should follow the same format.\n"
        "After generating summary for each round of conversation, generate an overall feedback on the entire dialogue. An example: Overall, the negotiator applied de-escalation strategies effectively, especially in capturing emotions and promoting collaboration. However, there is room for more explicit alignment with emotions and strategic focus at each stage to strengthen the negotiation process."
    )

    # Call GPT to generate feedback
    feedback = call_gpt(feedback_prompt,temp)

    if feedback == "error":
        print("An error occurred while generating feedback.")
    else:
        print("Here's the feedback for the negotiator based on the provided dialogue:\n")
        print(feedback)

        # save the feedback
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name = f"feedback_{timestamp}.txt"
        output_dir = f"{shared_folder_path}/feedback_logs/"
        os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, "w") as file:
            file.write(f"Dialogue:\n{input_dialogue}\n\nFeedback:\n{feedback}")

        print(f"\nFeedback has been saved as {file_name}.")

def call_gpt(gpt_prompt,temperature,api_key = openai_api_key):
    client = OpenAI(api_key=api_key)
    try:
        MODEL="gpt-4o"
        #MODEL = "gpt-3.5-turbo-1106"
        completion = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "user", "content": gpt_prompt}
          ],
          temperature = temperature
        )
        gpt_response = completion.choices[0].message.content
        return gpt_response
    except Exception as e:
        print(f"An error occurred: {e}")
        return "error"

def call_whisper(mp3_file_path, api_key = openai_api_key):
    client = OpenAI(api_key=api_key)
    try:
        audio_file = open(mp3_file_path, "rb")
        transcription = client.audio.transcriptions.create(
          model="whisper-1",
          file=audio_file,
          response_format="text"
        )
        return transcription
    except Exception as e:
        print(f"An error occurred during transcription: {e}")
        return "error"

def check_strategy(user_response, conversation_history, stage):
    initial_prompt = "You are a GPT model designed to evaluate if the user correctly applies specific de-escalation strategy. Here is a description of the 6 different stages the aggressive adversarial party could be at during a negotiation, and the correct strategy a professional negotiator should take for each stage:"

    # read in strategy
    with open(f"{shared_folder_path}/prompts/simulation_strategy.txt", "r") as file:
        content = file.read()
    strategy = "Here is a description of the 6 different stages the aggressive adversarial party could be at during a negotiation, and the correct strategy a professional negotiator should take for each stage, in order to de-escalate the situation and start to work towards common understanding and solutions:"
    strategy = (strategy + '\n'+ content)

    # sample script
    with open(f"{shared_folder_path}/prompts/simulation_sample_script.txt", "r") as file:
        content = file.read()
    sample_script = "Here is an example of the 6 stages negotiation. Use this as a referece when evaluating if the user correctly applies specific de-escalation techniques:"
    sample_script = (sample_script + '\n'+ content)

    # conversation history
    conversation_history_and_prompt = (
                f"This is the conversation history so far:\n{conversation_history}.\n"
                f"The latest negotiator's response in the conversation is: {user_response}, "
                f"Does the latest negotiator's response follow the stage {stage} de-escalation strategy? Please only output 1 if it follows the strategy, and output 0 if it does not follow."
            )

    gpt_prompt = initial_prompt + '\n' + strategy + '\n' + sample_script + '\n' + conversation_history_and_prompt
    gpt_output = call_gpt(gpt_prompt,temp)
    if gpt_output == '1':
        return 1
    elif gpt_output == '0':
        return 0
    else:
        return 'error'

def run_gpt_as_negotiator(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role):
    error = False
    conversation = []
    stages = []
    gpt_strategies = []
    continue_conv = True
    arrive_6 = False

    # initial_prompt
    with open(f"{shared_folder_path}/prompts/negotiator_initial_prompt.txt", "r") as file:
        initial_prompt = file.read()

    # read in strategy
    with open(f"{shared_folder_path}/prompts/simulation_strategy.txt", "r") as file:
        content = file.read()
    strategy = "Here is a description of the 6 different stages the aggressive adversarial party could be at during a negotiation, and the correct strategy a professional negotiator should take for each stage, in order to de-escalate the situation and start to work towards common understanding and solutions:"
    strategy = (strategy + '\n'+ content)

    # negotiation guidelines
    with open(f"{shared_folder_path}/prompts/negotiation_guidelines.txt", "r") as file:
        content = file.read()
    guideline = "The following is the negotiation guideline:"
    guideline = (guideline + '\n'+ content)

    # simulation
    stage = 0
    # write the simulation dialogue
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"simulation_log_{timestamp}.txt"
    output_dir = f"{shared_folder_path}/simulation_dialogues/"
    file_path = os.path.join(output_dir, file_name)
    with open(file_path, "w") as file:
        while continue_conv and len(stages) <= 15 and not error:
            if stage == 0:
                scenario_and_context = (
                    f"Now user has input a scenario: {simulation_description}\n"
                    f"User chose to play as {simulation_user_role}, which is {simulation_counterparty}. "
                    f"Now simulate a professional negotiator, who is {simulation_negotiator}, "
                    f"and start the conversation with an introduction. Here is a template: 'Welcome. Thank you for meeting me today. I understand you are the Student Union President, and I’m here on behalf of the university administration. I appreciate you coming to discuss these important matters.' Please only output the dialogue of the negotiator that you act without other explanation."
                )
            else:
                #print('\n')
                #print("Identified stage: "+str(stage))
                conversation_history = '\n'.join(conversation)
                scenario_and_context = (
                    f"Now user has input a scenario: {simulation_description}\n"
                    f"User chose to play as {simulation_user_role}, which is {simulation_counterparty}. "
                    f"This is the conversation history:\n{conversation_history}.\n"
                    f"Now simulate a professional negotiator, who is {simulation_negotiator}, "
                    f"and continue the above conversation using the correct strategy at stage {stage}. "\
                    "Please respond only with the dialogue in direct speech without mentioning or indicating any role or explanation. Then switch to a new line and only output the strategy number you used in the response (an integer from 1 to 6)."
                )

            # Negotiator response
            gpt_prompt = initial_prompt + '\n' + strategy + '\n' + guideline + '\n' + scenario_and_context
            gpt_response = call_gpt(gpt_prompt, temp)
            if stage == 0:
                print("Negotiator:")
                print(gpt_response)
            else:
                print("Negotiator:")
                print(gpt_response[:-1].strip())
                gpt_strategies.append(int(gpt_response[-1]))
            if gpt_response == "error":
                error = True
                break
            negotiator_response = f'Negotiator({simulation_negotiator}): ' + gpt_response
            file.write(f"Stage {stage} - Negotiator:\n{negotiator_response}\n")
            conversation.append(negotiator_response)

            # Counterparty response
            print("Adversarial party:")
            user_response = input()
            file.write(f"Adversarial party({simulation_counterparty}): {user_response}\n")
            conversation.append(f'Adversarial party({simulation_counterparty}): ' + user_response)

            # identify the stage of counterparty
            conversation_history = '\n'.join(conversation)
            stage = identify_stage(user_response, conversation_history)
            if stage == "error":
                error = True
                print("An error occurred while identifying the user's stage.")
                break
            elif stage != 6 and arrive_6:
                arrive_6 = False
            elif stage == 6 and arrive_6:
                continue_conv = False
            elif stage == 6:
                arrive_6 = True
            stages.append(stage)

    # Generate summary
    scenario_and_context = (
                f"Here is the negotiation scenario: {simulation_description}\n"
                f"User chose to play as {simulation_user_role}, which is {simulation_counterparty}. "
                f"Now you have finished a negotiation simulation, and this is the conversation history:\n{conversation_history}.\n"
            )

    with open(f"{shared_folder_path}/prompts/feedback_template_gpt.txt", "r") as file:
        sample_feedback = file.read()

    feedback_prompt = initial_prompt + '\n' + strategy + '\n' + scenario_and_context
    feedback_prompt += (
        "We recorded the identified stage of the adversarial party and corresponding selected strategy at each round of the conversation."
        "Analyze the conversation and offer summary on the GPT’s handling of the situation."
        "Here is the summary of their performance:\n"
    )
    for i, j in zip(stages, gpt_strategies):
        feedback_prompt += f"Identified stage: {i}\n Strategy: {j}\n"

    feedback_prompt += (
        "In the summary, list the strategies that GPT used at each stage of the negotiation."
        "Based on this performance, provide constructive summary by explaining why those strategies were selected."
        "Analyze the conversation on how to identify the user’s stage and emphasize the importance of the selected strategy."
        f"Here is an example summary for conversation 2, where the conversation is at stage 3: {sample_feedback}. You should follow the same format.\n"
        "After generating summary for each round of conversation, generate an overall summary on the entire dialogue. An example: Overall, the negotiation was handled effectively by identifying the stages of the adversarial party and applying the appropriate strategies. Each strategy was selected to de-escalate tensions and guide the conversation toward constructive solutions. By recognizing the emotions and demands of the student union and steering the dialogue toward collaboration and actionable plans, the negotiator contributed to a positive and productive outcome, aligning both parties toward shared goals."
    )
    feedback = call_gpt(feedback_prompt,temp)
    if stage == 6 and arrive_6:
        prefix_feedback = "Great job! You progressed through various stages up to the final stage, Stage 6. Here's a constructive summary for your reference:\n"
    else:
        prefix_feedback = "Good job! Although you didn’t make it through all six stages, you successfully completed 15 rounds of conversation. Here's a constructive summary for your reference:\n"
    print('\n')
    print(prefix_feedback)
    print(feedback)

    conversation.append(prefix_feedback + feedback)


    file_name = f"simulation_log_{timestamp}.txt"
    output_dir = f"{shared_folder_path}/simulation_dialogues/"
    file_path = os.path.join(output_dir, file_name)
    final_feedback = prefix_feedback + feedback
    with open(file_path, "a") as file:
        file.write("\nFinal Summary:\n")
        file.write(final_feedback + "\n")
    print(f"Simulation completed. Simulation script is saved as {file_name}")

def identify_stage(user_response, conversation_history):
    initial_prompt = "You are a GPT designed to identify the user’s stage during a high-pressure negotiation."

    # read in strategy
    with open(f"{shared_folder_path}/prompts/simulation_strategy.txt", "r") as file:
        content = file.read()
    strategy = "Here is a description of the 6 different stages the aggressive adversarial party could be at during a negotiation, and the correct strategy a professional negotiator should take for each stage, in order to de-escalate the situation and start to work towards common understanding and solutions:"
    strategy = (strategy + '\n'+ content)

    # sample script
    with open(f"{shared_folder_path}/prompts/simulation_sample_script.txt", "r") as file:
        content = file.read()
    sample_script = "Here is an example of the 6 stages of adversarial party in a negotiation. Use this as a reference when identifying the user’s stage"
    sample_script = (sample_script + '\n'+ content)

    # conversation history
    conversation_history_and_prompt = (
                f"This is the conversation history so far:\n{conversation_history}.\n"
                f"The latest adversarial party's response in the conversation is: {user_response}, "
                f"What is the latest stage of the adversarial party? Please only output a number from 1 to 6."
            )

    gpt_prompt = initial_prompt + '\n' + strategy + '\n' + sample_script + '\n' + conversation_history_and_prompt
    gpt_output = call_gpt(gpt_prompt,temp)
    valid_inputs = {'1', '2', '3', '4', '5', '6'}
    if gpt_output in valid_inputs:
        return int(gpt_output)
    else:
        return 'error'

# Interactive simulation practice

## GPT as the Adversarial Party

In [50]:
simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role = simulation_setup()

Hi, I'm a Negotiation Simulator. Please provide a description of the negotiation case:

Description of the negotiation case is successfully read in.
In this negotiation case, which side is the negotiator who wants to de-escalate and which side is the adversarial party who has strong demands or may have strong resistance? Just give me the two roles in the format A, B in the order negotiator, adversarial party:
University administration, Student union
Do you want to upload a dialogue for feedback or do you want to start a simulation? Reply 'Feedback' or 'Simulation'
Simulation
You choose to start a simulation.
In this negotiation case, which role do you want to play? Reply 'Negotiator' or 'Adversarial Party'
Negotiator
You choose to play as the Negotiator. The negotiation starts now.


In [53]:
run_simulation(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role)

Adversarial Party:
Thank you for meeting with us today. The situation on campus has spiraled out of control, and it's solely due to the administration's negligence. The rising tuition fees, abysmal housing conditions, and the ongoing threat to student safety have pushed us to our limits. We demand immediate action—no more delays or excuses. It's unacceptable, and we cannot take it anymore.
Negotiator:
[pause for 3 seconds] I hear your concerns. It seems that these previous meetings have only increased your frustration because there were no tangible results. I also understand that the arrests and the involvement of security forces have added to the strain. These issues are sensitive, especially when it involves your peers and colleagues. Let's discuss this together and work on finding meaningful solutions to address your concerns.
Adversarial Party:
We are done with empty words. We need immediate and concrete action from the administration, not more discussions or false promises. The ar

## GPT as the Negotiator

In [54]:
simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role = simulation_setup()

Hi, I'm a Negotiation Simulator. Please provide a description of the negotiation case:

Description of the negotiation case is successfully read in.
In this negotiation case, which side is the negotiator who wants to de-escalate and which side is the adversarial party who has strong demands or may have strong resistance? Just give me the two roles in the format A, B in the order negotiator, adversarial party:
University administration, Student union
Do you want to upload a dialogue for feedback or do you want to start a simulation? Reply 'Feedback' or 'Simulation'
Simulation
You choose to start a simulation.
In this negotiation case, which role do you want to play? Reply 'Negotiator' or 'Adversarial Party'
Adversarial Party
You choose to play as the Adversarial Party. The negotiation starts now.


In [57]:
run_simulation(simulation_description, simulation_negotiator, simulation_counterparty, simulation_user_role)

Negotiator:
Welcome. Thank you for meeting me today. I understand you are the Student Union President, and I’m here on behalf of the university administration. I appreciate you coming to discuss these important matters.
Adversarial party:
Thank you for listening, but we need more than just understanding. Students are under intense pressure, dealing with soaring tuition, unsafe housing, and unfair treatment. This situation is beyond unacceptable, and we cannot take it anymore. The administration’s lack of action has pushed us to this point. We are here for real, immediate changes, not another vague conversation.
Negotiator:
"I hear your frustrations and the urgency in your demands. Let's focus on the specific issues you've raised—tuition, housing, and treatment—and work together to find actionable solutions. We all share the goal of ensuring a safe and supportive environment for students. Can we identify the immediate steps we should consider?"
Adversarial party:
We've heard enough talk

# Negotiation Reocording Feedback

In [59]:
run_gpt_feedback(simulation_description, simulation_negotiator, simulation_counterparty)

Do you want to provide the negotiation dialogue as text or upload an MP3 recording? Reply 'Text' or 'MP3'.
MP3

You choose to provide the dialogue as MP3.
Please input the file path for the MP3 file:
/content/drive/My Drive/LLM_Negotiation_Simulator/prompts/example-recording-1.mp3
File path successfully uploaded.

Processing audio...

Here is the transcript of the dialogue you provided: 

Welcome, Mr. Mohamed. Thank you for coming. And thank you for meeting me today. I understand that you are the student union president or representative. And here I'm presenting the administration of university. Welcome. Thank you, Mr. Taha. Thank you for meeting us. You are aware of the situation on the campus has reached a kind of a critical point. This time around, we are not here for a round of empty promises. We've had several meetings with you, and it has been about empty promises and half measures. We, the student body, we have been patient for too long. And I think we cannot take it anymore. We